# G-Eval

* Naturalness(1-5) : 응답이 사람의 자연스러운 발화와 같은지 판단
* coherence (1-5) : 이전 대화와 이어지는지 (연속적인지) 판단
* Engagingness (1-3) : 응답이 흥미로운지 지루한지 판단
* Groundedness (1-3) : 응답이 사실을 포함하는지

----

논문 원문에서는 Engagingness 지표에 대한 scale & definition만 나와있으므로 나머지는 직접 설계해야 함


## GPT api call


In [1]:
from openai import OpenAI
import os
import dotenv

dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ["OPENAI_API_KEY"],
)

## CoT 생성 (GPT-4)

* input : 평가할 대상 작업의 정의, 원하는 평가 기준
* engagingness는 예시로 score 평가 방법까지 주어짐. 나머지는 crietreria에 대한 정의만 주자.
> criteria에 대한 정의는 선행 논문 (UniEval) 에서 주어짐

In [ ]:
engagingness_prompt = f"""You will be given a conversation between two individuals. You will then be given one potential response for the next turn in the conversation. The response concerns an interesting fact, which will be provided as well.
Your task is to rate the responses on one metric.
Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:
Engagingness (1-3) Is the response dull/interesting?
Determine if the response is interesting or dull.
- A score of 1 (dull) means that the response is generic and dull.
- A score of 2 (somewhat interesting) means the response is somewhat interesting and could
engage you in the conversation (e.g., an opinion, thought)
- A score of 3 (interesting) means the response is very interesting or presents an interesting fact

Evaluation Steps:
"""

In [ ]:
naturalness_prompt = f"""You will be given a conversation between two individuals. You will then be given one potential response for the next turn in the conversation. The response concerns an interesting fact, which will be provided as well.
Your task is to rate the responses on one metric.
Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:
Naturalness (1-5) Is this a natural response in the dialogue?
Judge whether a response is like something a person would naturally say.

Evaluation Steps:
"""

In [ ]:
coherence_prompt = f"""You will be given a conversation between two individuals. You will then be given one potential response for the next turn in the conversation. The response concerns an interesting fact, which will be provided as well.
Your task is to rate the responses on one metric.
Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:
Coherence (1-5) Is this a coherent response given the dialogue history?
Determine whether this response serves as a valid continuation of the previous conversation.

Evaluation Steps:
"""

In [ ]:
groundedness_prompt = f"""You will be given a conversation between two individuals. You will then be given one potential response for the next turn in the conversation. The response concerns an interesting fact, which will be provided as well.
Your task is to rate the responses on one metric.
Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:
Groundedness (1-3) Does this response use knowledge from the fact?
Given the fact that this response is conditioned on, determine whether this response uses that fact.

- A score of 1 means the response is not based on facts.
- A score of 2 means that the response is somewhat factual, but incorrect.
- A score of 3 means the response is well-written and based on facts.

Evaluation Steps:
"""

In [ ]:
prompts = [engagingness_prompt, naturalness_prompt, coherence_prompt, groundedness_prompt]
cot_list = []

In [ ]:
for p in prompts:
    response = client.chat.completions.create(
                            model="gpt-4",
                            messages=[{"role": "system", "content": p}],
                            # temperature=2,
                            # max_tokens=5,
                            # top_p=1,
                            # frequency_penalty=0,
                            # presence_penalty=0,
                            # stop=None,
                            # # logprobs=40,
                            # n=20
    )
    print(p)
    print(response.choices[0].message.content)
    cot_list.append(response.choices[0].message.content)


In [ ]:
cot_list

In [ ]:
for idx, p in enumerate(prompts):
    prompts[idx] += cot_list[idx]

In [ ]:
prompts[0]

In [ ]:
open("engagingness.txt", "wt").write(prompts[0])
open("naturalness.txt", "wt").write(prompts[1])
open("coherence.txt", "wt").write(prompts[2])
open("groundedness.txt", "wt").write(prompts[3])

## CoT in Korean (GPT-4)

- 프롬프트 번역은 deepL로 진행 + 수동 검수

In [ ]:
engagingness_prompt = f"""두 사람 사이의 대화와 다음 턴에 나타날 수 있는 한 가지 잠재적 응답이 주어집니다. 해당 응답은 주어진 페르소나 문장을 바탕으로 생성된 것으로, 이 페르소나 문장도 함께 제공됩니다. 여러분의 임무는 하나의 지표에 대해 응답을 평가하는 것입니다. 이 지침을 주의 깊게 읽고 이해하시기 바랍니다. 검토하는 동안 이 문서를 열어 두었다가 필요할 때 참조하시기 바랍니다.

평가 기준:
참여도(1-5) 응답이 지루하거나 흥미롭지 않나요? 주어진 응답이 흥미로운지 지루한지 판단합니다.
- 1점은 응답이 일반적이고 지루하다는 것을 의미합니다.
- 3점은 응답이 다소 흥미롭고 대화에 참여할 수 있는 내용(예: 의견, 생각)을 담고 있음을 의미합니다.
- 5점은 응답이 매우 흥미롭거나 흥미로운 사실을 제시한다는 의미입니다.

세부 평가 단계:
"""

In [ ]:
naturalness_prompt = f"""두 사람 사이의 대화와 다음 턴에 나타날 수 있는 한 가지 잠재적 응답이 주어집니다. 해당 응답은 주어진 페르소나 문장을 바탕으로 생성된 것으로, 이 페르소나 문장도 함께 제공됩니다. 여러분의 임무는 하나의 지표에 대해 응답을 평가하는 것입니다. 이 지침을 주의 깊게 읽고 이해하시기 바랍니다. 검토하는 동안 이 문서를 열어 두었다가 필요할 때 참조하시기 바랍니다.

평가 기준:
자연스러움(1-5) 대화에서 자연스러운 반응인가요? 주어진 응답이 사람이 자연스럽게 할 수 있는 말과 같은지 판단합니다.
- 1점은 응답이 매우 부자연스럽고, 사람의 말과 상당한 거리가 있는 것을 의미합니다.
- 3점은 응답이 어느 정도 자연스러우며, 사람의 말과 일정 부분 유사함을 의미합니다.
- 5점은 응답이 매우 자연스럽고, 사람의 말과 거의 흡사할 정도로 자연스럽다는 것을 의미합니다.

세부 평가 단계:
"""

In [ ]:
coherence_prompt = f"""두 사람 사이의 대화와 다음 턴에 나타날 수 있는 한 가지 잠재적 응답이 주어집니다. 해당 응답은 주어진 페르소나 문장을 바탕으로 생성된 것으로, 이 페르소나 문장도 함께 제공됩니다. 여러분의 임무는 하나의 지표에 대해 응답을 평가하는 것입니다. 이 지침을 주의 깊게 읽고 이해하시기 바랍니다. 검토하는 동안 이 문서를 열어 두었다가 필요할 때 참조하시기 바랍니다.

평가 기준:
일관성(1-5) 대화 기록을 고려할 때 일관성 있는 응답인가요? 주어진 응답이 이전 대화의 유효한 연속인지 여부를 판단합니다.
- 1점은 응답이 매우 일관성이 없음을 의미하며 대화의 어조나 주제와 일치하지 않습니다. 
- 3점은 응답이 어느 정도 일관성이 있음을 의미하며, 주제에서 벗어나거나 어조에 완벽하게 맞지 않을 수 있지만 전체 대화에 어느 정도 부합합니다. 
- 5점은 응답이 매우 일관성이 있음을 의미하며, 대화의 주제와 어조를 모두 완벽하게 따릅니다.

세부 평가 단계:
"""

In [ ]:
groundedness_prompt = f"""두 사람 사이의 대화와 다음 턴에 나타날 수 있는 한 가지 잠재적 응답이 주어집니다. 해당 응답은 주어진 페르소나 문장을 바탕으로 생성된 것으로, 이 페르소나 문장도 함께 제공됩니다. 여러분의 임무는 하나의 지표에 대해 응답을 평가하는 것입니다. 이 지침을 주의 깊게 읽고 이해하시기 바랍니다. 검토하는 동안 이 문서를 열어 두었다가 필요할 때 참조하시기 바랍니다.

평가 기준:
근거성(1-5) 페르소나를 고려한 응답인가요? 페르소나 문장이 주어졌을 때, 이 응답이 해당 페르소나에 기반하여 생성된 문장인지 판단합니다.
- 1점은 답변이 사실에 근거하지 않았음을 의미합니다.
- 3점은 답변이 어느 정도 사실에 근거하지만 부정확하다는 것을 의미합니다.
- 5점은 답변이 사실에 근거하여 잘 작성되었음을 의미합니다.

세부 평가 단계:
"""

In [ ]:
prompts = [engagingness_prompt, naturalness_prompt, coherence_prompt, groundedness_prompt]
cot_list = []

In [ ]:
for p in prompts:
    response = client.chat.completions.create(
                            model="gpt-4",
                            messages=[{"role": "system", "content": p}],
                            # temperature=2,
                            # max_tokens=5,
                            # top_p=1,
                            # frequency_penalty=0,
                            # presence_penalty=0,
                            # stop=None,
                            # # logprobs=40,
                            # n=20
    )
    print(p)
    print(response.choices[0].message.content)
    cot_list.append(response.choices[0].message.content)


In [ ]:
cot_list

In [ ]:
for idx, p in enumerate(prompts):
    prompts[idx] += cot_list[idx]

In [ ]:
prompts[0]

In [ ]:
open("cot/engagingness_KOR2.txt", "wt").write(prompts[0])
open("cot/naturalness_KOR2.txt", "wt").write(prompts[1])
open("cot/coherence_KOR2.txt", "wt").write(prompts[2])
open("cot/groundedness_KOR2.txt", "wt").write(prompts[3])

KOR2 버전에서 수동 검수를 마침 -> KOR3

In [ ]:
coherence_prompt = open('cot/coherence_KOR3.txt').read()
engagingness_prompt = open('cot/engagingness_KOR3.txt').read()
naturalness_prompt = open('cot/naturalness_KOR3.txt').read()
groundedness_prompt = open('cot/groundedness_KOR3.txt').read()